# Part 1


* Your stakeholder only wants you to include information for movies based on the following specifications:

* Exclude any movie with missing values for genre or runtime

* Include only full-length movies (titleType = "movie").

* Include only fictional movies (not from documentary genre)

* Include only movies that were released 2000 - 2021 (include 2000 and 2021)

* Include only movies that were released in the United States

## Imports and Data

In [1]:
#imports

import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# getting urls

basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz' 

In [3]:
#storing urls

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep ='\t', low_memory = False)
ratings = pd.read_csv(ratings_url, sep = '\t', low_memory = False)


## Basics Data and Cleaning

In [4]:
# Exploring df

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
#Checking types

basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250673 entries, 0 to 9250672
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 635.2+ MB


In [6]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
basics.replace({'\\N':np.nan}, inplace = True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [7]:
# checking missing values
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1233635
endYear           9154163
runtimeMinutes    6772962
genres             427882
dtype: int64

In [8]:
# almost all of endYear is missing so i;m going to drop it
basics = basics.drop(columns = 'endYear')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...
9250668,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,NaN,"Action,Drama,Family"
9250669,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,NaN,"Action,Drama,Family"
9250670,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,NaN,"Action,Drama,Family"
9250671,tt9916856,short,The Wind,The Wind,0,2015,27,Short


In [9]:
#Dropping na columns
basics.dropna(inplace = True) 
basics.isna().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
dtype: int64

In [10]:
#checking for duplicate rows
basics.duplicated().sum()

0

In [11]:
# looking for movies
is_movie = basics['titleType'] == 'movie'
is_movie

0          False
1          False
2          False
3          False
4          False
           ...  
9250623     True
9250629    False
9250664    False
9250671    False
9250672    False
Name: titleType, Length: 2373824, dtype: bool

In [12]:
#Changing start year dtype from object to int 
basics['startYear'] = basics['startYear'].astype(int)
#filtering out movies earlier than 2000
basics = basics.loc[(basics['startYear'] >=2000) & (basics['startYear'] <=2022) & is_movie]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,133,Documentary
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
66309,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,47,Documentary
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
...,...,...,...,...,...,...,...,...
9250438,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"
9250522,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,123,Drama
9250563,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary
9250590,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary


In [13]:
# get data type of column 'startYear'
dataType = basics.dtypes['startYear']
print('Data type of each column startYear in the Dataframe :')
print(dataType)

Data type of each column startYear in the Dataframe :
int32


In [14]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
77934,tt0079644,movie,November 1828,November 1828,0,2001,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
9250345,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9250354,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
9250393,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
9250438,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


In [15]:
#checking info is consistent
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143270 entries, 34792 to 9250522
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          143270 non-null  object
 1   titleType       143270 non-null  object
 2   primaryTitle    143270 non-null  object
 3   originalTitle   143270 non-null  object
 4   isAdult         143270 non-null  object
 5   startYear       143270 non-null  int32 
 6   runtimeMinutes  143270 non-null  object
 7   genres          143270 non-null  object
dtypes: int32(1), object(7)
memory usage: 9.3+ MB


## AKAS Data and Cleaning

In [16]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [17]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33280943 entries, 0 to 33280942
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


In [18]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
akas.replace({'\\N':np.nan}, inplace = True)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [19]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     5
region              1863183
language            6270153
types              27986080
attributes         33033661
isOriginalTitle        2187
dtype: int64

In [20]:
# dropping remainder of nans
akas.dropna(inplace = True)
akas.isna().sum()

titleId            0
ordering           0
title              0
region             0
language           0
types              0
attributes         0
isOriginalTitle    0
dtype: int64

In [21]:
#checking for duplicates
akas.duplicated().sum()

0

In [22]:
# checking unique values
akas.nunique()

titleId            435
ordering            38
title              451
region               5
language             4
types                3
attributes          38
isOriginalTitle      1
dtype: int64

In [23]:
#showing only movies made in the usa
is_US_Movie = akas['region'].str.contains('US',case=False)
akas = akas[is_US_Movie]
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
100767,tt0022542,1,Di shtime fun Yisroel,US,yi,alternative,YIVO translation,0
112670,tt0024265,4,Geleb un gelakht,US,yi,alternative,modern translation,0
116051,tt0024751,9,Avram Ovenu,US,yi,alternative,YIVO translation,0
124651,tt0026010,3,Der yidishe Kenigen Lir,US,yi,alternative,YIVO translation,0
138084,tt0027911,1,Libe un Laydnshaft,US,yi,alternative,modern translation,0
145682,tt0028902,4,Freylekhe kabtsonim,US,yi,alternative,YIVO translation,0
146156,tt0028957,4,Grine Felder,US,yi,alternative,YIVO translation,0
146610,tt0029013,3,Di heylige Shvue,US,yi,alternative,YIVO translation,0
146804,tt0029042,3,Ikh vil zayn a Mame,US,yi,alternative,YIVO translation,0
152389,tt0029765,3,Vu iz mayn Kind?,US,yi,alternative,YIVO translation,0


In [24]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
basics



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
77934,tt0079644,movie,November 1828,November 1828,0,2001,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
9250345,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9250354,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
9250393,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
9250438,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


In [25]:
#checking data is consistent
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 100767 to 1027255
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          22 non-null     object
 1   ordering         22 non-null     int64 
 2   title            22 non-null     object
 3   region           22 non-null     object
 4   language         22 non-null     object
 5   types            22 non-null     object
 6   attributes       22 non-null     object
 7   isOriginalTitle  22 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.5+ KB


## Ratings Data and Cleaning

In [26]:
# Getting overview of the data
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1911
1,tt0000002,5.8,257
2,tt0000003,6.5,1716
3,tt0000004,5.6,169
4,tt0000005,6.2,2532


In [27]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260817 entries, 0 to 1260816
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1260817 non-null  object 
 1   averageRating  1260817 non-null  float64
 2   numVotes       1260817 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB


In [28]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
ratings.replace({'\\N':np.nan}, inplace = True)
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [29]:
# checking duplicates
ratings.duplicated().sum()

0

In [30]:
# Filter the basics table down to only include the US by using the filter ratings dataframe
keepers2 =basics['tconst'].isin(ratings['tconst'])
basics





,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
77934,tt0079644,movie,November 1828,November 1828,0,2001,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
9250345,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9250354,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
9250393,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
9250438,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


## Saving Data as csv


In [31]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv('Data/title.akas.csv.gz',compression = 'gzip', index=False)
ratings.to_csv('Data/title.ratings.csv.gz', compression = 'gzip', index=False)

# Part 2 

Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

In [32]:
import os, time, json
import tmdbsimple as tmdb 
with open('C:/Users/Sean/.edit/tmdb_api.json', 'r') as f:
    login = json.load(f)
from tqdm.notebook import tqdm_notebook
## Display the keys of the loaded dict
login.keys()

# Credential import
tmdb.API_KEY =  login['api-key']


In [33]:
#saving api call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 '.ipynb_checkpoints',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2001.csv.gz']

In [34]:
def get_movie_certification(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == "US":
            info['certifcation'] = c['certification']     
    return info

In [35]:
# function for json file creation

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [36]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

basics


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
143265,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
143266,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
143267,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
143268,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


In [37]:
#defining a list of years
YEARS_TO_GET = [2000,2001]

# defining an errors list
errors = [ ]

## Loop Extraction

In [38]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
#Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()    
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
 #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
 # Retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)  
 # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
# Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip",
                     index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2717 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2846 [00:00<?, ?it/s]

In [39]:
#any movie ids that caused an error?
print(f"- Total errors: {len(errors)}") 

- Total errors: 5563


Load in your csv.gz's of results for each year extracted.
Concatenate the data into 1 dataframe for the remainder of the analysis.
Once you have your data from the API, they would like you to perform some light EDA to show:
1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
* Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
2. How many movies are there in each of the certification categories (G/PG/PG-13/R)?
3. What is the average revenue per certification category?
4. What is the average budget per certification category?



In [40]:
#fetching data
movies_2k = pd.read_csv('Data/tmdb_api_results_2000.json', low_memory = False)
movies_2k

,"[{""imdb_id"": 0}]"
